In [2]:
from AutoEncoder import myAutoEncoder
import tensorflow as tf
import numpy as np
import pandas as pd

## Load Data

In [3]:
df = pd.read_csv('/Users/bowen/Desktop/H/deeplearninveststrat/Data/100_clean.csv')
df = df.drop('Unnamed: 0', axis=1)
df['Date'] = pd.to_datetime(df['Date'])

In [4]:
featureList = ['Revenue', 'Cost of Goods Sold', 'SG&A','EBIT', 'Net Income', 
               'Cash and Cash Equivalents', 'Recievables','Inventories', 'Other Current Assets', 
               'PP&E', 'Other Assets', 'Debt in Current Liabilities', 'Accounts Payable', 'Taxes Payable',
               'Other Current Liabilities', 'Total Liabilities']
quarterlyData = {}
dfgb = df.groupby('gvkey')
for gvkey, data in dfgb:
    if len(data) == 115:
        quarterlyData[gvkey] = data.iloc[:114][featureList]

## Denoising with AutoEncoder

In [5]:
def Run(Data):
    n_input = 16
    n_hidden_1 = 11
    n_hidden_2 = 4
    
    AutoEncoder = myAutoEncoder(n_input, n_hidden_1, n_hidden_2)
    X_train, trainDataWithNoise, testData = AutoEncoder.NormalizedStockData(Data)
    
    batch_size = 6
    learning_rate = 0.05
    training_epochs = 150

    X = tf.placeholder("float", [batch_size, n_input])
    y_true = tf.placeholder("float", [batch_size, n_input])
    encoder_op = AutoEncoder.encoder(X)
    decoder_op = AutoEncoder.decoder(encoder_op)
    y_pred = decoder_op
    cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)

        for epoch in range(training_epochs):
            row = 0
            for i in range(int(trainDataWithNoise.shape[0] / batch_size)):
                batch_xs = trainDataWithNoise[row:row + batch_size]
                y_xs = X_train[row:row + batch_size]
                row += batch_size
                _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs, y_true:y_xs })
            print("Epoch:", '%04d' % (epoch + 1), "cost =", "{:.9f}".format(c))
        print("Optimization Finished!")

        result = []
        row = 0
        for i in range(int(testData.shape[0] / batch_size)):
            batch_xs = testData[row:row + batch_size]
            row += batch_size
            res = sess.run(decoder_op, feed_dict={X: batch_xs})
            result.extend(res.tolist())
    return result


def transToDataFrame(result, gvkey, df):
    r = pd.DataFrame(result)
    r.columns = ['Revenue', 'Cost of Goods Sold', 'SG&A','EBIT', 'Net Income', 
               'Cash and Cash Equivalents', 'Recievables','Inventories', 'Other Current Assets', 
               'PP&E', 'Other Assets', 'Debt in Current Liabilities', 'Accounts Payable', 'Taxes Payable',
               'Other Current Liabilities', 'Total Liabilities']
    
    r['timestemp'] = df[df['gvkey'] == gvkey][:-1]['Date'].values
    r['EBIT/EV'] = df[df['gvkey'] == gvkey][:-1]['EBIT/EV'].values
    r['gevkey'] = str(gvkey)
    return r

## test for single stock

In [9]:
data = quarterlyData[1209]

In [10]:
result = Run(data)

Epoch: 0001 cost = 0.272305042
Epoch: 0002 cost = 0.204936311
Epoch: 0003 cost = 0.184733197
Epoch: 0004 cost = 0.184265241
Epoch: 0005 cost = 0.174867257
Epoch: 0006 cost = 0.158710197
Epoch: 0007 cost = 0.135040894
Epoch: 0008 cost = 0.108944856
Epoch: 0009 cost = 0.087852716
Epoch: 0010 cost = 0.070899174
Epoch: 0011 cost = 0.060787380
Epoch: 0012 cost = 0.053644612
Epoch: 0013 cost = 0.034826767
Epoch: 0014 cost = 0.040307406
Epoch: 0015 cost = 0.053147793
Epoch: 0016 cost = 0.033799674
Epoch: 0017 cost = 0.039009970
Epoch: 0018 cost = 0.034449946
Epoch: 0019 cost = 0.035440553
Epoch: 0020 cost = 0.036877315
Epoch: 0021 cost = 0.034457568
Epoch: 0022 cost = 0.043006089
Epoch: 0023 cost = 0.036133587
Epoch: 0024 cost = 0.035262648
Epoch: 0025 cost = 0.122273542
Epoch: 0026 cost = 0.035823286
Epoch: 0027 cost = 0.053133231
Epoch: 0028 cost = 0.035420269
Epoch: 0029 cost = 0.038425915
Epoch: 0030 cost = 0.036349487
Epoch: 0031 cost = 0.036896620
Epoch: 0032 cost = 0.035665382
Epoch: 0

In [11]:
transToDataFrame(result,1209, df)

,Revenue,Cost of Goods Sold,SG&A,EBIT,Net Income,Cash and Cash Equivalents,Recievables,Inventories,Other Current Assets,PP&E,Other Assets,Debt in Current Liabilities,Accounts Payable,Taxes Payable,Other Current Liabilities,Total Liabilities,timestemp,EBIT/EV,gevkey
0,0.024978,0.018352,0.067439,0.500363,0.458592,0.195587,0.021696,0.066963,0.100297,0.044508,0.062604,0.097989,0.025332,0.076149,0.210249,0.039281,1990-03-31,0.028919,1209
1,0.025450,0.018639,0.068341,0.501662,0.460556,0.196369,0.022236,0.068251,0.101020,0.045312,0.063372,0.099126,0.025788,0.077230,0.213025,0.040079,1990-06-30,0.025749,1209
2,0.025732,0.018827,0.068852,0.502433,0.461626,0.196768,0.022563,0.069008,0.101518,0.045818,0.063839,0.099790,0.026064,0.077844,0.214403,0.040588,1990-09-30,0.032678,1209
3,0.026312,0.019180,0.070005,0.503995,0.463898,0.197611,0.023242,0.070636,0.102421,0.046822,0.064816,0.101227,0.026661,0.079156,0.217281,0.041590,1990-12-31,0.027214,1209
4,0.026647,0.019389,0.070666,0.504884,0.465139,0.198049,0.023637,0.071577,0.102968,0.047413,0.065388,0.102056,0.027014,0.079896,0.218732,0.042184,1991-03-31,0.024855,1209
5,0.026216,0.019116,0.069825,0.503737,0.463549,0.197488,0.023128,0.070371,0.102253,0.046649,0.064653,0.100995,0.026562,0.078948,0.216875,0.041415,1991-06-30,0.024122,1209
6,0.029403,0.021148,0.076209,0.511880,0.474292,0.200920,0.027015,0.079537,0.107650,0.052440,0.070270,0.109002,0.030136,0.085798,0.227280,0.047300,1991-09-30,0.021611,1209
7,0.030389,0.021771,0.078270,0.514247,0.477261,0.201741,0.028269,0.082481,0.109308,0.054267,0.072066,0.111541,0.031338,0.087865,0.229390,0.049180,1991-12-31,0.022384,1209
8,0.030438,0.021813,0.078332,0.514358,0.477366,0.201768,0.028332,0.082605,0.109426,0.054373,0.072154,0.111647,0.031391,0.087947,0.229416,0.049291,1992-03-31,0.021757,1209
9,0.032161,0.022963,0.081720,0.518284,0.482019,0.202974,0.030570,0.087681,0.112484,0.057669,0.075300,0.115975,0.033524,0.091379,0.231947,0.052722,1992-06-30,0.021165,1209


## Denoising all stocks

In [14]:
finalResult = {}
gvkeyList = [i for i in quarterlyData.keys()]
for gvkey in gvkeyList:
    result = Run(quarterlyData[gvkey])
    finalResult[gvkey] = transToDataFrame(result,gvkey, df)

Epoch: 0001 cost = 0.323617905
Epoch: 0002 cost = 0.234526992
Epoch: 0003 cost = 0.197402462
Epoch: 0004 cost = 0.175037682
Epoch: 0005 cost = 0.155463263
Epoch: 0006 cost = 0.135655105
Epoch: 0007 cost = 0.108079612
Epoch: 0008 cost = 0.080102555
Epoch: 0009 cost = 0.059005696
Epoch: 0010 cost = 0.048085991
Epoch: 0011 cost = 0.041462470
Epoch: 0012 cost = 0.035169750
Epoch: 0013 cost = 0.036986355
Epoch: 0014 cost = 0.043455571
Epoch: 0015 cost = 0.035004620
Epoch: 0016 cost = 0.037808217
Epoch: 0017 cost = 0.039526675
Epoch: 0018 cost = 0.032330647
Epoch: 0019 cost = 0.036933661
Epoch: 0020 cost = 0.037872627
Epoch: 0021 cost = 0.047842760
Epoch: 0022 cost = 0.049492065
Epoch: 0023 cost = 0.046948064
Epoch: 0024 cost = 0.038684756
Epoch: 0025 cost = 0.095325761
Epoch: 0026 cost = 0.035030495
Epoch: 0027 cost = 0.042304490
Epoch: 0028 cost = 0.040988341
Epoch: 0029 cost = 0.039173335
Epoch: 0030 cost = 0.038217306
Epoch: 0031 cost = 0.038204599
Epoch: 0032 cost = 0.037507210
Epoch: 0

Epoch: 0120 cost = 0.011679106
Epoch: 0121 cost = 0.012743910
Epoch: 0122 cost = 0.011616942
Epoch: 0123 cost = 0.013150800
Epoch: 0124 cost = 0.011943869
Epoch: 0125 cost = 0.012630455
Epoch: 0126 cost = 0.011903559
Epoch: 0127 cost = 0.012394679
Epoch: 0128 cost = 0.011696129
Epoch: 0129 cost = 0.012227747
Epoch: 0130 cost = 0.011676613
Epoch: 0131 cost = 0.012288715
Epoch: 0132 cost = 0.011708990
Epoch: 0133 cost = 0.012390274
Epoch: 0134 cost = 0.011726973
Epoch: 0135 cost = 0.012688863
Epoch: 0136 cost = 0.011779991
Epoch: 0137 cost = 0.013256487
Epoch: 0138 cost = 0.011512953
Epoch: 0139 cost = 0.012520685
Epoch: 0140 cost = 0.011713889
Epoch: 0141 cost = 0.012490782
Epoch: 0142 cost = 0.011974707
Epoch: 0143 cost = 0.012425852
Epoch: 0144 cost = 0.012344033
Epoch: 0145 cost = 0.012441055
Epoch: 0146 cost = 0.014069900
Epoch: 0147 cost = 0.013887469
Epoch: 0148 cost = 0.013076416
Epoch: 0149 cost = 0.011794488
Epoch: 0150 cost = 0.011604198
Optimization Finished!
Epoch: 0001 cost

Epoch: 0103 cost = 0.007037129
Epoch: 0104 cost = 0.008387051
Epoch: 0105 cost = 0.008460685
Epoch: 0106 cost = 0.008588814
Epoch: 0107 cost = 0.008482787
Epoch: 0108 cost = 0.008216466
Epoch: 0109 cost = 0.007945965
Epoch: 0110 cost = 0.007655922
Epoch: 0111 cost = 0.007549835
Epoch: 0112 cost = 0.007320283
Epoch: 0113 cost = 0.007351903
Epoch: 0114 cost = 0.007149993
Epoch: 0115 cost = 0.007286360
Epoch: 0116 cost = 0.007484661
Epoch: 0117 cost = 0.007715814
Epoch: 0118 cost = 0.007879814
Epoch: 0119 cost = 0.034344036
Epoch: 0120 cost = 0.006700512
Epoch: 0121 cost = 0.019051388
Epoch: 0122 cost = 0.006484393
Epoch: 0123 cost = 0.006965688
Epoch: 0124 cost = 0.007842381
Epoch: 0125 cost = 0.006985831
Epoch: 0126 cost = 0.007357836
Epoch: 0127 cost = 0.006930260
Epoch: 0128 cost = 0.007482493
Epoch: 0129 cost = 0.007067692
Epoch: 0130 cost = 0.010005831
Epoch: 0131 cost = 0.007545003
Epoch: 0132 cost = 0.010924106
Epoch: 0133 cost = 0.007754685
Epoch: 0134 cost = 0.007742118
Epoch: 0

Epoch: 0067 cost = 0.028966969
Epoch: 0068 cost = 0.027038159
Epoch: 0069 cost = 0.026936861
Epoch: 0070 cost = 0.025113940
Epoch: 0071 cost = 0.025508611
Epoch: 0072 cost = 0.023558527
Epoch: 0073 cost = 0.024438107
Epoch: 0074 cost = 0.022565760
Epoch: 0075 cost = 0.023198932
Epoch: 0076 cost = 0.021912068
Epoch: 0077 cost = 0.021691119
Epoch: 0078 cost = 0.022188514
Epoch: 0079 cost = 0.020546185
Epoch: 0080 cost = 0.022734025
Epoch: 0081 cost = 0.019431874
Epoch: 0082 cost = 0.020810507
Epoch: 0083 cost = 0.020152882
Epoch: 0084 cost = 0.019358804
Epoch: 0085 cost = 0.021511296
Epoch: 0086 cost = 0.017767478
Epoch: 0087 cost = 0.021357194
Epoch: 0088 cost = 0.017955855
Epoch: 0089 cost = 0.019924689
Epoch: 0090 cost = 0.017761590
Epoch: 0091 cost = 0.019018346
Epoch: 0092 cost = 0.017530024
Epoch: 0093 cost = 0.017300108
Epoch: 0094 cost = 0.020644406
Epoch: 0095 cost = 0.016473318
Epoch: 0096 cost = 0.020692199
Epoch: 0097 cost = 0.019141508
Epoch: 0098 cost = 0.016388424
Epoch: 0

Epoch: 0042 cost = 0.009697891
Epoch: 0043 cost = 0.013775057
Epoch: 0044 cost = 0.007057483
Epoch: 0045 cost = 0.011321475
Epoch: 0046 cost = 0.008714396
Epoch: 0047 cost = 0.008070429
Epoch: 0048 cost = 0.009318077
Epoch: 0049 cost = 0.007689921
Epoch: 0050 cost = 0.008312425
Epoch: 0051 cost = 0.009437772
Epoch: 0052 cost = 0.006681157
Epoch: 0053 cost = 0.009866524
Epoch: 0054 cost = 0.006351979
Epoch: 0055 cost = 0.008776583
Epoch: 0056 cost = 0.008125939
Epoch: 0057 cost = 0.008044396
Epoch: 0058 cost = 0.007065120
Epoch: 0059 cost = 0.009975979
Epoch: 0060 cost = 0.005421137
Epoch: 0061 cost = 0.008637314
Epoch: 0062 cost = 0.007059805
Epoch: 0063 cost = 0.006092236
Epoch: 0064 cost = 0.008725702
Epoch: 0065 cost = 0.007550752
Epoch: 0066 cost = 0.005810694
Epoch: 0067 cost = 0.009011618
Epoch: 0068 cost = 0.005106032
Epoch: 0069 cost = 0.006681935
Epoch: 0070 cost = 0.007686567
Epoch: 0071 cost = 0.005505911
Epoch: 0072 cost = 0.007554298
Epoch: 0073 cost = 0.008727834
Epoch: 0

...

Epoch: 0098 cost = 0.013718500
Epoch: 0099 cost = 0.012509153
Epoch: 0100 cost = 0.013981611
Epoch: 0101 cost = 0.012206907
Epoch: 0102 cost = 0.014356702
Epoch: 0103 cost = 0.012027711
Epoch: 0104 cost = 0.014095847
Epoch: 0105 cost = 0.012216034
Epoch: 0106 cost = 0.012702701
Epoch: 0107 cost = 0.012324587
Epoch: 0108 cost = 0.013124692
Epoch: 0109 cost = 0.012396728
Epoch: 0110 cost = 0.012968659
Epoch: 0111 cost = 0.012448994
Epoch: 0112 cost = 0.012968752
Epoch: 0113 cost = 0.012366824
Epoch: 0114 cost = 0.013178427
Epoch: 0115 cost = 0.012494181
Epoch: 0116 cost = 0.013730059
Epoch: 0117 cost = 0.013399400
Epoch: 0118 cost = 0.013522723
Epoch: 0119 cost = 0.014059099
Epoch: 0120 cost = 0.013044287
Epoch: 0121 cost = 0.013754354
Epoch: 0122 cost = 0.012075871
Epoch: 0123 cost = 0.014092677
Epoch: 0124 cost = 0.012045923
Epoch: 0125 cost = 0.013238042
Epoch: 0126 cost = 0.012228587
Epoch: 0127 cost = 0.012228042
Epoch: 0128 cost = 0.012443658
Epoch: 0129 cost = 0.012533451
Epoch: 0

Epoch: 0079 cost = 0.014254221
Epoch: 0080 cost = 0.014194581
Epoch: 0081 cost = 0.013858638
Epoch: 0082 cost = 0.013540373
Epoch: 0083 cost = 0.013719392
Epoch: 0084 cost = 0.013816149
Epoch: 0085 cost = 0.013898410
Epoch: 0086 cost = 0.014664103
Epoch: 0087 cost = 0.014823664
Epoch: 0088 cost = 0.014604646
Epoch: 0089 cost = 0.015290885
Epoch: 0090 cost = 0.016840899
Epoch: 0091 cost = 0.014730182
Epoch: 0092 cost = 0.013414645
Epoch: 0093 cost = 0.014323760
Epoch: 0094 cost = 0.012326743
Epoch: 0095 cost = 0.012737665
Epoch: 0096 cost = 0.012992017
Epoch: 0097 cost = 0.012482460
Epoch: 0098 cost = 0.011591851
Epoch: 0099 cost = 0.013978516
Epoch: 0100 cost = 0.012972292
Epoch: 0101 cost = 0.013056689
Epoch: 0102 cost = 0.013033181
Epoch: 0103 cost = 0.011396013
Epoch: 0104 cost = 0.011951822
Epoch: 0105 cost = 0.013596900
Epoch: 0106 cost = 0.011586584
Epoch: 0107 cost = 0.011602148
Epoch: 0108 cost = 0.011403475
Epoch: 0109 cost = 0.010185714
Epoch: 0110 cost = 0.011768617
Epoch: 0

Epoch: 0046 cost = 0.033508603
Epoch: 0047 cost = 0.029672349
Epoch: 0048 cost = 0.041152392
Epoch: 0049 cost = 0.030009245
Epoch: 0050 cost = 0.034188103
Epoch: 0051 cost = 0.029096762
Epoch: 0052 cost = 0.029987797
Epoch: 0053 cost = 0.028087646
Epoch: 0054 cost = 0.028178474
Epoch: 0055 cost = 0.027110094
Epoch: 0056 cost = 0.029751843
Epoch: 0057 cost = 0.028409891
Epoch: 0058 cost = 0.042915687
Epoch: 0059 cost = 0.030972064
Epoch: 0060 cost = 0.037068505
Epoch: 0061 cost = 0.028937660
Epoch: 0062 cost = 0.028889343
Epoch: 0063 cost = 0.027886210
Epoch: 0064 cost = 0.027336599
Epoch: 0065 cost = 0.026351241
Epoch: 0066 cost = 0.027110064
Epoch: 0067 cost = 0.026134433
Epoch: 0068 cost = 0.036373965
Epoch: 0069 cost = 0.029786944
Epoch: 0070 cost = 0.041728377
Epoch: 0071 cost = 0.028465644
Epoch: 0072 cost = 0.029245367
Epoch: 0073 cost = 0.026943117
Epoch: 0074 cost = 0.027129412
Epoch: 0075 cost = 0.026002802
Epoch: 0076 cost = 0.025330162
Epoch: 0077 cost = 0.024843404
Epoch: 0

Epoch: 0022 cost = 0.032396890
Epoch: 0023 cost = 0.033740085
Epoch: 0024 cost = 0.036735419
Epoch: 0025 cost = 0.033177596
Epoch: 0026 cost = 0.035962489
Epoch: 0027 cost = 0.031877887
Epoch: 0028 cost = 0.030864825
Epoch: 0029 cost = 0.030954173
Epoch: 0030 cost = 0.029555762
Epoch: 0031 cost = 0.029269909
Epoch: 0032 cost = 0.028799767
Epoch: 0033 cost = 0.027879933
Epoch: 0034 cost = 0.028203050
Epoch: 0035 cost = 0.026810111
Epoch: 0036 cost = 0.027678952
Epoch: 0037 cost = 0.026133856
Epoch: 0038 cost = 0.027190091
Epoch: 0039 cost = 0.025688857
Epoch: 0040 cost = 0.026732132
Epoch: 0041 cost = 0.025304353
Epoch: 0042 cost = 0.026404744
Epoch: 0043 cost = 0.025000453
Epoch: 0044 cost = 0.026006980
Epoch: 0045 cost = 0.024813905
Epoch: 0046 cost = 0.025895497
Epoch: 0047 cost = 0.024715627
Epoch: 0048 cost = 0.025771340
Epoch: 0049 cost = 0.024848802
Epoch: 0050 cost = 0.025694484
Epoch: 0051 cost = 0.024874499
Epoch: 0052 cost = 0.025947953
Epoch: 0053 cost = 0.025064511
Epoch: 0

Epoch: 0139 cost = 0.068980537
Epoch: 0140 cost = 0.045214076
Epoch: 0141 cost = 0.038243018
Epoch: 0142 cost = 0.037850697
Epoch: 0143 cost = 0.035766631
Epoch: 0144 cost = 0.034930170
Epoch: 0145 cost = 0.034899443
Epoch: 0146 cost = 0.035635352
Epoch: 0147 cost = 0.042813748
Epoch: 0148 cost = 0.063622884
Epoch: 0149 cost = 0.066311061
Epoch: 0150 cost = 0.050535250
Optimization Finished!


In [15]:
finalResult = pd.concat([v.assign(gvkey=k) for k, v in finalResult.items()], ignore_index=True)

In [17]:
finalResult.to_csv('DenoisngData.csv')